## Data Engineering: Boston Crime Database with Postgres

For this project, we will be creating a crime database with the `boston.csv` text file. The purpose is to demonstrate the ability to create a database and table within the Postgres as well as creating user with the least-privilidge permissions necessary for that user. The database to be created will be stored within the `crimes` schema that will also be created to store the tables.

In [3]:
# establish packages

import pandas as pd
import csv
import psycopg2
import numpy as np

In [41]:
# connect to the dataquest postgres database

conn = psycopg2.connect(dbname = 'dq', user='dq')
cur = conn.cursor()
# turn auto commit on to establish 
conn.autocommit = True
cur.execute('CREATE DATABASE crime_db')
conn.autocommit = False
conn.close()

SyntaxError: syntax error at or near "NOT"
LINE 1: CREATE DATABASE IF NOT EXISTS crime_db
                           ^


In [28]:
# connect to crime_db, this database will be use throuhout the rest of the database 
conn2 = psycopg2.connect(dbname = 'crime_db', user = 'dq')
cur2 = conn2.cursor()

# create schema for crimes_db
cur2.execute('CREATE SCHEMA crimes;')
conn2.commit()

In [19]:
"""analyze the boston.csv file to understand the datatypes needed to create 
the postgres table to which it will be stored """

with open('boston.csv') as file:
    reader = list(csv.reader(file))
    header = reader[0]
    first_row = reader[1]

print(header)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [20]:
# get python value set

def get_col_set(csv_file, col_index):
    values = set()
    with open(csv_file) as file:
        next(file)
        reader = csv.reader(file)
        for row in reader:
            values.add(row[col_index])
    return values
    
for i in range(len(header)):
    values = get_col_set('boston.csv', i)
    print(i, header[i], len(values), sep='\t')
        
       

0	incident_number	298329
1	offense_code	219
2	description	239
3	date	1177
4	day_of_the_week	7
5	lat	18177
6	long	18177


In [21]:
col_values = get_col_set('boston.csv', 2)
max_value = 0
for i in col_values:
    if len(i) > max_value:
        max_value = len(i)
max_value

58

After determining the index for the description column, we looped through the descriptions column to determine the the max length for each distinct value available for the this column. The max character value count was determined to be 58. We have determined the appropriate variables to create the postgres table within our `crimes_db`

### Creating tables within the Crimes Schema 

We will create the following enumerate type for the weekdays seeing that it will consist of no more than 7 distinct values. With those values Wednesday has larger total of characters in a given value. To do that, we used the get_col_set and will recall the header as well as first row of our csv file.

In [9]:
print(header)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [10]:
col_values = get_col_set('boston.csv', 4)
max_value_week = 0
for i in col_values:
    if len(i) > max_value_week:
        max_value_week = len(i)
print(col_values)

{'Wednesday', 'Tuesday', 'Saturday', 'Monday', 'Friday', 'Thursday', 'Sunday'}


In [25]:
cur2.execute("CREATE TYPE Weekdays AS ENUM ('Monday', 'Saturday', 'Friday', 'Thursday', 'Tuesday', 'Sunday', 'Wednesday')")

-------------------------------------
| Tables | Type Description |
------------|---------|
| incident_number | INT PRIMARY KEY
| offense_code	| INT
| description |	TEXT
| date  |	DATE
| day_of_the_week | Weekdays
| lat | Decimal
| long| Decimal

In [32]:
# Create database within the crimes schema
cur2.execute("rollback;")
cur2.execute("""CREATE TABLE IF NOT EXISTS crimes.boston_crimes ( 
                            incident_number INTEGER PRIMARY KEY,
                            offense_code INTEGER,
                            description varchar(100),
                            date DATE,
                            day_of_the_week Weekdays,
                            lat DECIMAL,
                            long DECIMAL );""")

conn2.commit()


### Importing `boston.csv` data into our new `boston_crimes` table

Now that we have created our boston_crimes table within the crimes schema, it is now time to import the `boston.csv`. There are a few options to be used for this step. However it is suggested to use the .copy_exper module to perform this process as it is efficient compared to iterating over each line to copy into the database 

example of import

`with open("filename.csv") as f:
    cur.copy_expert("COPY table_name FROM STDIN WITH CSV HEADER;", f) `


In [36]:
with open('boston.csv') as f: 
    cur2.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [61]:
# Review loaded data by performing SELECT statement

cur2.execute("SELECT * FROM crimes.boston_crimes;")
cur2.fetchmany(5)

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943')),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.33418175'),
  Decimal('-71.07866441')),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.27536542'),
  Decimal('-71.09036101'))]

## Database and Table Permissions

The next step in establishing our database, tables and schemas are to establish permissions amongst users. To do so, we will utilize both the `GRANT` and `REVOKE` commands. The most efficient method to grant or revoke permission for several current and future users is to create user groups. We can establish permissions of the user groups and will add users to those groups to automatically establish what access the user has for Databases and Tables.

#### First step to prevent inadvertently retrieving priviliges from the public user group

In [43]:
# Revoke all public group access from the public schema in Postgres
cur2.execute("REVOKE ALL ON SCHEMA public from public;")

# Revoke all access from the crime_db database on the schema group
cur2.execute("REVOKE ALL ON DATABASE crime_db from public;")

#### Next, create user groups and provide grant and revoke permissions

**readonly:** group intended to perform `SELECT` queries on the following database

**readwrite:** group that has the ability to perform -- `SELECT, INSERT, DELETE and UPDATE queries.`

In [44]:
# create read only group
cur2.execute("CREATE GROUP readonly NOLOGIN;")

# create read write group
cur2.execute("CREATE GROUP readwrite NOLOGIN;")

In [47]:
# Grant connection Access to the readwrite and readonly group
cur2.execute("GRANT CONNECT ON  DATABASE crime_db to readonly;")
cur2.execute("GRANT CONNECT ON DATABASE crime_db to readwrite;")

# Grant USAGE on the crimes SCHEMA
cur2.execute("GRANT USAGE on SCHEMA crimes to readonly;")
cur2.execute("GRANT USAGE on SCHEMA crimes to readwrite;")

In [49]:
#Grant SELECT access to all tables in Crimes schema for readonly

cur2.execute("GRANT SELECT on ALL TABLES IN SCHEMA crimes to readonly;")

# Grant SELECT, INSERT, DELETE and UPDATE on all tables in Crimes Schema for readwrite
cur2.execute("GRANT SELECT, INSERT, DELETE,UPDATE on ALL TABLES IN SCHEMA crimes to readwrite;")

#### Finally, create users and assign to the appropriate user groups created

users to create: 

- `data_analyst`
- `data_scientist`

In [50]:
# create data analyst user
cur2.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")

#assign data analyst to readonly user group
cur2.execute("GRANT readonly to data_analyst;")

# create data scientist user
cur2.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")

#assign data scientist to readwrite user group
cur2.execute("GRANT readwrite to data_scientist;")

### Test configuration and updated user permissions

Now that we have configured the appropriate groups and users, we will review if those updates were made by using the following queries for `readonly` and `readwrite`


In [59]:
# review users with 'readwrite access'
cur2.execute("""SELECT grantor, grantee, table_name, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';""")
cur2.fetchall()




[('dq', 'readwrite', 'boston_crimes', 'INSERT'),
 ('dq', 'readwrite', 'boston_crimes', 'SELECT'),
 ('dq', 'readwrite', 'boston_crimes', 'UPDATE'),
 ('dq', 'readwrite', 'boston_crimes', 'DELETE')]

In [60]:
# review users with 'readonly access'
cur2.execute("""SELECT grantor, grantee, table_name, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly';""")
cur2.fetchall()


[('dq', 'readonly', 'boston_crimes', 'SELECT')]